In [ ]:
import cv2
import numpy as np
import dlib
import pandas as pd
import os, glob
from imutils import face_utils
import argparse
import imutils

In [ ]:
# Load the detector
detector = dlib.get_frontal_face_detector()

In [ ]:
# Load the predictor
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

In [7]:
def file_features(cap):
    
    import pandas as pd
    import cv2
    import dlib
    import numpy as np
    from imutils import face_utils
    import os, glob
    import argparse
    import imutils
    
    # Load the detector
    detector = dlib.get_frontal_face_detector()
    
    # Load the predictor
    predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

    emotions_features = pd.DataFrame()
    
#     for filepath in glob.glob(cap):

    # Read the video
#         img = cv2.imread(filepath)
#         cap = cv2.VideoCapture(path)
    ret, frame = cap.read()

    # Convert image into grayscale
    gray = cv2.cvtColor(src=frame, code=cv2.COLOR_BGR2GRAY)

    # Use detector to find landmarks
    faces = detector(gray)

    for face in faces:
        x1 = face.left() # left point
        y1 = face.top() # top point
        x2 = face.right() # right point
        y2 = face.bottom() # bottom point


        landmarks = predictor(image=gray, box=face)

        emptyarray = []

        for n in range(0, 68):
            x = landmarks.part(n).x
            y = landmarks.part(n).y
#             print(x)
            # Created a coordinate numpy array
            coordinates = np.array([x, y])

            # Append coordinate values (x, y) to emptyarray
            emptyarray.append(coordinates[0])
            emptyarray.append(coordinates[1])

            # Create dataframe
            df = pd.DataFrame({'features':[emptyarray]})

        # Append all df from all files/faces
        emotions_features = emotions_features.append(df)

    emotions_features['features']= emotions_features['features'].astype(str).str[1:-1]
    emotions_features2 = emotions_features['features'].str.split(",", n = 135, expand = True) 
    emotions_features2.rename(columns= lambda s:f"data{s}", inplace=True )

    emotions_features2 = emotions_features2.apply(pd.to_numeric)
    emotions_features2.fillna(0,inplace=True)
    emotions_features2 = emotions_features2.astype({"data0": int})

    
    emotions_features2.reset_index(drop=True, inplace=True)

    return emotions_features2

In [8]:
def make_prediction(model, cap):
    
    # Extract image file features
    df = file_features(cap)
    
    # Make prediction using imported model
    prediction = model.predict_classes(df)
#     prediction_label = label_encoder.inverse_transform(prediction)
    
    # Emotion labels
    emotion_codes = {
                0: 'Angry',
                1:'Disgust' ,
                2:'Fear',
                3:'Happy',
                4:'Sad',
                5:'Surprise',
                6:'Neutral'
               }
    result = emotion_codes[prediction[0]]
    
    return result   

In [17]:
def realtime_face_detection(canny=False, resize=False, default=True):
    import cv2
    import numpy as np
    import dlib
    import pandas as pd
    import os, glob
    from imutils import face_utils
    import argparse
    import imutils
    from tensorflow.keras.models import load_model
    model =load_model("image_emotions_model_test.h5")
    
    # Load the detector
    detector = dlib.get_frontal_face_detector()
    
    # Load the predictor
    predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")
    
#     tracker = cv2.TrackerMIL_create()


    # read the image
    cap = cv2.VideoCapture(0)

    while True:

        ok, frame = cap.read()
#         print(ok)
        
        if canny:    
            # Canny edge detection (optimization)
            frame = cv2.Canny(frame, 120, 160)
            
            # Use detector to find landmarks
            faces = detector(frame)
            
        if resize:
            # Get image size info
            height, width, channels = frame.shape
            
            # Resize image/frame for optimization
            frame = imutils.resize(frame, height=120, width=160)
            #     print (height, width, channels)
        if default:
            # Convert image into grayscale
            gray = cv2.cvtColor(src=frame, code=cv2.COLOR_BGR2GRAY)

            # Use detector to find landmarks
            faces = detector(gray)
        


        for face in faces:
            x1 = face.left()  # left point
            y1 = face.top()  # top point
            x2 = face.right()  # right point
            y2 = face.bottom()  # bottom point
#             print(face)
            
            if default:
                # Create landmark object
                landmarks = predictor(image=gray, box=face)
#                 ok = tracker.init(gray, face)

            
            if canny:
                # Create landmark object
                landmarks = predictor(image=frame, box=face)

            # Loop through all the points
            for n in range(0, 68):
                x = landmarks.part(n).x
                y = landmarks.part(n).y

                # Draw a circle
                cv2.circle(img=frame, center=(x, y), radius=3, color=(0, 255, 0), thickness=-1)

            # Make Prediction
            emotion_prediction = make_prediction(model, cap)

            # show the emotion prediction if value was retrieved
            if emotion_prediction:
        #         cv2.putText(frame, emotion_prediction, (x1 - 10, y1 - 10),
                cv2.putText(frame, emotion_prediction, (10, 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)




        # show the image
        cv2.imshow(winname="Face", mat=frame)

        # Exit when escape is pressed
        if cv2.waitKey(delay=1) == 27:
            break

    # When everything done, release the video capture and video write objects
    cap.release()

    # Close all windows
    cv2.destroyAllWindows()

In [18]:
realtime_face_detection()